In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, concat_ws, split, hour, to_date, dayofmonth, desc
import re

from utils import get_session

spark = get_session()

df = spark.read.csv('hdfs:///tweets/debate-tweets-001.tsv', sep='\t').select('_c1', '_c7', '_c8')
df.show()

Setting default log level to "

22/12/08 22:43:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
+--------------------+--------------------+----------+
|                 _c1|                 _c7|       _c8|
+--------------------+--------------------+----------+
|@anacddd verdade,...|Wed Oct 15 14:31:...|2014-10-15|
|              Que ñ*|Wed Oct 15 14:31:...|2014-10-15|
| Vou quebrar a Bruna|Wed Oct 15 14:31:...|2014-10-15|
|agora vou p segun...|Wed Oct 15 14:31:...|2014-10-15|
|Me sinto tão bem ...|Wed Oct 15 14:31:...|2014-10-15|
|Eu estou aqui, de...|Wed Oct 15 14:31:...|2014-10-15|
|Quando vai embora...|Wed Oct 15 14:31:...|2014-10-15|
|@paynecaralhudo k...|Wed Oct 15 14:31:...|2014-10-15|
|Conceição da Barr...|Wed Oct 15 14:31:...|2014-10-15|
| @Maniavato te amo ♥|Wed Oct 15 14:31:...|2014-10-15|
|Alg me curtindo rs ♡|Wed Oct 15 14:31:...|2014-10-15|
|@MiiluAA No, porq...|Wed Oct 15 14:31:...|2014-10-15|
|#EMABiggestFansJu...|Wed Oct 15

In [2]:
from time import strptime

#Get date and UTC time columns to create a Timestamp column
df2 = df.withColumn('datetime',to_timestamp(concat_ws(' ',df._c8, split(df._c7, ' ')[3])))

#Exclude all datetimes with null values and group by day and time
df3 = df2.filter(df2.datetime.isNotNull()) \
      .groupBy(to_date(df2.datetime),hour(df2.datetime)).count() \
      .withColumnRenamed('to_date(datetime)','date') \
      .withColumnRenamed('hour(datetime)', 'hour')

df4 = df3.orderBy(desc('date'), desc('hour'))
df4.show()

+----------+----+------+
|      date|hour| count|
+----------+----+------+
|2014-10-20|   7|  4922|
|2014-10-20|   6| 14504|
|2014-10-20|   5| 29497|
|2014-10-20|   4| 57005|
|2014-10-20|   3| 92015|
|2014-10-20|   2|124997|
|2014-10-20|   1|134555|
|2014-10-20|   0|123610|
|2014-10-19|  23| 95083|
|2014-10-19|  22| 86629|
|2014-10-19|  21| 77033|
|2014-10-19|  20| 75453|
|2014-10-19|  19| 77265|
|2014-10-19|  18| 80192|
|2014-10-19|  17| 73534|
|2014-10-19|  16| 76299|
|2014-10-19|  15| 75866|
|2014-10-19|  14| 69949|
|2014-10-19|  13| 55888|
|2014-10-19|  12| 38047|
+----------+----+------+
only showing top 20 rows



In [3]:
spark.stop()